# Model Server

In [7]:
import nuclio

In [8]:
%nuclio config kind = "nuclio"

%nuclio: setting kind to 'nuclio'


In [9]:
%%nuclio env

FEATURES_TABLE = /User/demos/network-operations/data/ 
#streaming/features
PREDICTIONS_TABLE = /User/demos/network-operations/streaming/predictions
prediction_col = predictions
# model_path = /User/demos/network-operations/artifacts/model/2/model.pkl
model_path = store://network-operations/train_model#f2ea18dd-504d-4d14-83d2-a326fe5afc37
model_name = netops_predictor_v1
model_class = MLRunModel
model_col = model
model_class_col = class
worker_col = worker
hostname_col = hostname
timestamp_col = when
orig_timestamp_col = timestamp

%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting 'PREDICTIONS_TABLE' environment variable
%nuclio: setting 'prediction_col' environment variable
%nuclio: setting '# model_path' environment variable
%nuclio: setting 'model_path' environment variable
%nuclio: setting 'model_name' environment variable
%nuclio: setting 'model_class' environment variable
%nuclio: setting 'model_col' environment variable
%nuclio: setting 'model_class_col' environment variable
%nuclio: setting 'worker_col' environment variable
%nuclio: setting 'hostname_col' environment variable
%nuclio: setting 'timestamp_col' environment variable
%nuclio: setting 'orig_timestamp_col' environment variable


%nuclio: cannot find "=" in line


## Function

In [10]:
# nuclio: start-code

In [11]:
import os
import pandas as pd
import cloudpickle
import numpy as np
import json
from mlrun import get_or_create_ctx
import socket

In [12]:
def get_data_parquet(context):
    mpath = [os.path.join(context.features_table, file) for file in os.listdir(context.features_table) if file.endswith(('parquet', 'pq'))]
    files_by_updated = sorted(mpath, key=os.path.getmtime, reverse=True)
    return pd.read_parquet(files_by_updated[:1][0])

In [19]:
def save_to_parquet(context, df: pd.DataFrame):
    print('Saving features to Parquet')
    
    # Need to fix timestamps from ns to ms if we write to parquet 
    # And add this model name to indexes
    keys = list([name if name != context.orig_timestamp_col else context.timestamp_col for name in df.index.names]) + [context.model_col, context.model_class_col, context.worker_col, context.hostname_col]
    df = df.reset_index()
    df[context.timestamp_col] = df.pop(context.orig_timestamp_col).astype('datetime64[ms]')
    
    # Fix indexes
    df = df.set_index(keys)
    
    # Save parquet
    first_timestamp = df.index[0][0].strftime('%Y%m%dT%H%M%S')
    last_timestamp = df.index[-1][0].strftime('%Y%m%dT%H%M%S')
    filename = first_timestamp + '-' + last_timestamp + '.parquet'
    filepath = os.path.join(context.predictions_table, filename)
    with open(filepath, 'wb+') as f:
        df.to_parquet(f)

In [20]:
def init_context(context):
    # Set vars from env
    setattr(context, 'model_name', os.getenv('model_name', 'netops_model'))
    setattr(context, 'model_col', os.getenv('model_col', 'model'))
    setattr(context, 'model_class_col', os.getenv('model_class_col', 'class'))
    setattr(context, 'worker_col', os.getenv('worker_col', 'worker'))
    setattr(context, 'hostname_col', os.getenv('hostname_col', 'hostname'))
    setattr(context, 'timestamp_col', os.getenv('timestamp_col', 'when'))
    setattr(context, 'orig_timestamp_col', os.getenv('orig_timestamp_col', 'timestamp'))
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    setattr(context, 'predictions_table', os.getenv('PREDICTIONS_TABLE', 'netops_predictions'))
    setattr(context, 'prediction_col', os.getenv('prediction_col', 'prediction'))
    
    # Load model
    model_path = os.environ['model_path']
    if model_path.startswith('store://'):
        mlctx = get_or_create_ctx('inference')
        model = mlctx.get_dataitem(model_path)
        model_path = os.path.join(model.url, 'model.pkl')
    with open(model_path, 'rb') as f:
        model = cloudpickle.load(f)
    setattr(context, 'model', model)
    setattr(context, 'model_class', type(model).__name__)
    
     # Create saving directory if needed
    filepath = os.path.join(context.predictions_table)
    if not os.path.exists(filepath):
        os.makedirs(filepath)

In [21]:
def handler(context, event):
    
    if getattr(event.trigger, 'kind', 'cron') == 'cron':
        # Get latest parquets
        df = pd.read_parquet('/User/demos/network-operations/artifacts/selected_features.parquet').iloc[:, :-1] #get_data_parquet(context)

        # Predict
        df[context.prediction_col] = context.model.predict(df.values)
        
        # Add server metadata
        df[context.model_col] = context.model_name
        df[context.model_class_col] = context.model_class
        df[context.worker_col] = context.worker_id
        df[context.hostname_col] = socket.gethostname()

        # Save
        save_to_parquet(context, df)
    else:
        body = json.loads(event.body)
        feats = np.asarray(body['instances'])
        result: np.ndarray = context.model.predict(feats)
        return result.tolist()

In [22]:
# nuclio: end-code

## Local test

In [23]:
init_context(context)

> 2020-08-10 10:50:29,470 [info] logging run results to: http://mlrun-api:8080


/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [24]:
event = nuclio.Event(body='', trigger={'kind': 'cron'})
out = handler(context, event)
out

Saving features to Parquet


## Test

In [166]:
from mlrun import code_to_function, mount_v3io

In [167]:
fn = code_to_function('inference-server',
                      kind='nuclio',
                      project='network-operations')
fn.spec.base_spec['spec']['build']['baseImage'] = 'mlrun/ml-models'
fn.apply(mount_v3io())
fn.add_trigger('cron', nuclio.triggers.CronTrigger(interval='1m'))

In [168]:
fn.save()
fn.export('../src/inference-server.yaml')

> 2020-08-09 13:09:30,711 [info] function spec saved to path: ../src/inference-server.yaml


In [169]:
fn.deploy(project='network-operations')

> 2020-08-09 13:09:30,717 [info] deploy started
[nuclio] 2020-08-09 13:09:37,916 (info) Build complete
[nuclio] 2020-08-09 13:09:47,072 done updating network-operations-inference-server, function address: 192.168.224.209:32279


'http://192.168.224.209:32279'

In [25]:
pd.read_parquet('../streaming/predictions/20200630T064217-20200630T074212.parquet')

cpu_utilization  \
when                    company                     data_center      device        model               class                  worker hostname                                    
2020-06-30 06:42:17.454 Wilson_LLC                  Zachary_Drives   6001003522699 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        66.939139   
                                                    Obrien_Mountain  0966571261270 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        72.492707   
                                                                     8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.116878   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        64.944107   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        78.641128   
...                                                                                                                                                                        ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Obrien_Mountain  8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        59.574487   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd       100.000000   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd       100.000000   
                                                    Dominique_Branch 4579248894449 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.053014   
                                                                     7731097392608 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.629988   

                                                                                                                                                                  latency  \
when                    company                     data_center      device        model               class                  worker hostname                               
2020-06-30 06:42:17.454 Wilson_LLC                  Zachary_Drives   6001003522699 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.537279   
                                                    Obrien_Mountain  0966571261270 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
                                                                     8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    2.606934   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    1.571046   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
...                                                                                                                                                                   ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Obrien_Mountain  8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd  100.000000   
                                                                     4285071567351 netops_predictor_v1 RandomFores